# Import Packages

In [ ]:
import os

import torch
import torchvision
from torch.autograd import Variable

from architecture import MNIST_autoencoder as mnist
from functionalities import filemanager as fm
from functionalities import dataloader as dl
from functionalities import gpu 
from functionalities import plot as pl
from functionalities import trainer as tr
from tqdm import tqdm_notebook as tqdm

# Pretraining Setup

In [ ]:
num_epoch = 100
batch_size = 128
lr_init = 1e-3
milestones = [10 * x for x in range(1, 11)]
latent_dim_lst = [1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64]
number_dev = 0
get_model = mnist.mnist_autoencoder
modelname = "mnist_classic_bottleneck"

device = gpu.get_device(number_dev)
print(device)

In [ ]:
trainset, testset, classes = dl.load_mnist()
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, batch_size)

# Training

In [ ]:
tr.train_bottleneck_classic(num_epoch, get_model, modelname, milestones, latent_dim_lst, trainloader,
                     validloader, testloader, lr_init=lr_init, device=device, mnist=True)

# Plot Reconstruction and Difference Images Examples

In [ ]:
import matplotlib.pyplot as plt

num_img = 100
grid_row_size = 10

img, label = next(iter(testloader))
img = img.view(img.size(0), -1)
img = Variable(img).cuda()

for i in latent_dim_lst:
    print('bottleneck dimension: {}'.format(i))
    model = fm.load_model('{}_{}'.format(modelname, i))
    output = model(img)

    
    original = pl.to_img(img.cpu().data, [1, 28, 28]) 
    pic = pl.to_img(output.cpu().data, [1, 28, 28])

    print("Original Image:")
    pl.imshow(torchvision.utils.make_grid(original[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_original".format(i))
    print("Reconstructed Image:")
    pl.imshow(torchvision.utils.make_grid(pic[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_reconstructed".format(i))
    print("Difference:")
    diff_img = (original - pic + 1) / 2
    pl.imshow(torchvision.utils.make_grid(diff_img[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_difference".format(i))
    

# Plot Recontruction Loss against Bottleneck Size

In [ ]:
train, test = fm.load_variable("{}_bottleneck".format(modelname))
y = [train, test]
x = []
for loss in y:
    x.append([1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64])

pl.plot(latent_dim_lst, y, 'bottleneck size', 'loss', ['train', 'test'], 'Train & Test Reconstruction Loss History', 'loss_l1_bottleneck') 

# Repeat for classic 1024

In [ ]:
# Pretraining Setup
num_epoch = 100
batch_size = 128
lr_init = 1e-3
milestones = [10 * x for x in range(1, 11)]
latent_dim_lst = [1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64]
number_dev = 0
get_model = mnist.mnist_autoencoder_1024
modelname = "mnist_classic_1024_bottleneck"

device = gpu.get_device(number_dev)
print(device)

trainset, testset, classes = dl.load_mnist()
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, batch_size)

# Training
tr.train_bottleneck_classic(num_epoch, get_model, modelname, milestones, latent_dim_lst, trainloader,
                     validloader, testloader, lr_init=lr_init, device=device, mnist=True)

# Plot Reconstruction and Difference Images Examples
import matplotlib.pyplot as plt

num_img = 100
grid_row_size = 10

img, label = next(iter(testloader))
img = img.view(img.size(0), -1)
img = Variable(img).cuda()

for i in latent_dim_lst:
    print('bottleneck dimension: {}'.format(i))
    model = fm.load_model('{}_{}'.format(modelname, i))
    output = model(img)

    
    original = pl.to_img(img.cpu().data, [1, 28, 28]) 
    pic = pl.to_img(output.cpu().data, [1, 28, 28])

    print("Original Image:")
    pl.imshow(torchvision.utils.make_grid(original[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_original".format(i))
    print("Reconstructed Image:")
    pl.imshow(torchvision.utils.make_grid(pic[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_reconstructed".format(i))
    print("Difference:")
    diff_img = (original - pic + 1) / 2
    pl.imshow(torchvision.utils.make_grid(diff_img[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_difference".format(i))
    
# Plot Recontruction Loss against Bottleneck Size
train, test = fm.load_variable("{}_bottleneck".format(modelname))
y = [train, test]
x = []
for loss in y:
    x.append([1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64])

pl.plot(latent_dim_lst, y, 'bottleneck size', 'loss', ['train', 'test'], 'Train & Test Reconstruction Loss History', 'loss_l1_bottleneck') 

# Repeat for classic 2048

In [ ]:
# Pretraining Setup
num_epoch = 100
batch_size = 128
lr_init = 1e-3
milestones = [10 * x for x in range(1, 11)]
latent_dim_lst = [1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64]
number_dev = 0
get_model = mnist.mnist_autoencoder_2048
modelname = "mnist_classic_2048_bottleneck"

device = gpu.get_device(number_dev)
print(device)

trainset, testset, classes = dl.load_mnist()
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, batch_size)

# Training
tr.train_bottleneck_classic(num_epoch, get_model, modelname, milestones, latent_dim_lst, trainloader,
                     validloader, testloader, lr_init=lr_init, device=device, mnist=True)

# Plot Reconstruction and Difference Images Examples
import matplotlib.pyplot as plt

num_img = 100
grid_row_size = 10

img, label = next(iter(testloader))
img = img.view(img.size(0), -1)
img = Variable(img).cuda()

for i in latent_dim_lst:
    print('bottleneck dimension: {}'.format(i))
    model = fm.load_model('{}_{}'.format(modelname, i))
    output = model(img)

    
    original = pl.to_img(img.cpu().data, [1, 28, 28]) 
    pic = pl.to_img(output.cpu().data, [1, 28, 28])

    print("Original Image:")
    pl.imshow(torchvision.utils.make_grid(original[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_original".format(i))
    print("Reconstructed Image:")
    pl.imshow(torchvision.utils.make_grid(pic[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_reconstructed".format(i))
    print("Difference:")
    diff_img = (original - pic + 1) / 2
    pl.imshow(torchvision.utils.make_grid(diff_img[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_difference".format(i))
    
# Plot Recontruction Loss against Bottleneck Size
train, test = fm.load_variable("{}_bottleneck".format(modelname))
y = [train, test]
x = []
for loss in y:
    x.append([1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64])

pl.plot(latent_dim_lst, y, 'bottleneck size', 'loss', ['train', 'test'], 'Train & Test Reconstruction Loss History', 'loss_l1_bottleneck') 

# Repeat for deep classic

In [ ]:
# Pretraining Setup
num_epoch = 100
batch_size = 128
lr_init = 1e-3
milestones = [10 * x for x in range(1, 11)]
latent_dim_lst = [1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64]
number_dev = 0
get_model = mnist.mnist_autoencoder_deep_1024
modelname = "mnist_classic_deep_bottleneck"

device = gpu.get_device(number_dev)
print(device)

trainset, testset, classes = dl.load_mnist()
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, batch_size)

# Training
tr.train_bottleneck_classic(num_epoch, get_model, modelname, milestones, latent_dim_lst, trainloader,
                     validloader, testloader, lr_init=lr_init, device=device, mnist=True)

# Plot Reconstruction and Difference Images Examples
import matplotlib.pyplot as plt

num_img = 100
grid_row_size = 10

img, label = next(iter(testloader))
img = img.view(img.size(0), -1)
img = Variable(img).cuda()

for i in latent_dim_lst:
    print('bottleneck dimension: {}'.format(i))
    model = fm.load_model('{}_{}'.format(modelname, i))
    output = model(img)

    
    original = pl.to_img(img.cpu().data, [1, 28, 28]) 
    pic = pl.to_img(output.cpu().data, [1, 28, 28])

    print("Original Image:")
    pl.imshow(torchvision.utils.make_grid(original[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_original".format(i))
    print("Reconstructed Image:")
    pl.imshow(torchvision.utils.make_grid(pic[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_reconstructed".format(i))
    print("Difference:")
    diff_img = (original - pic + 1) / 2
    pl.imshow(torchvision.utils.make_grid(diff_img[:num_img].detach(), grid_row_size), filename="com_classic_mnist_{}_difference".format(i))
    
# Plot Recontruction Loss against Bottleneck Size
train, test = fm.load_variable("{}_bottleneck".format(modelname))
y = [train, test]
x = []
for loss in y:
    x.append([1, 2, 3, 4, 6, 8, 12, 16, 24, 32, 48, 64])

pl.plot(latent_dim_lst, y, 'bottleneck size', 'loss', ['train', 'test'], 'Train & Test Reconstruction Loss History', 'loss_l1_bottleneck') 